### Bytes literal & Encoding

- Send and Receive bytes

In [ ]:
b"hello world" == "hello world".encode("utf-8") # default encoding scheme is utf-8
bytes_array = b"hello world"
# for b in bytes_array:
#     print(f"{b:0b}", end="")

bytes_array.decode()

'hello world'

### Socket Programming with Python

A **socket address** is an end-point for data communication. It consists of an ip address and a port number

Development of application programs that makes use of these socket address to communicate is called **socket/network programming**.

The most common type for socket applications are client-server applications, where we have a server part and a client part. As the name suggests:
- the server part wait for requests from the clients
- clients request services from the servers.

### Server Program
To code an iterative server program in Python, we carry out the following in steps:
1. import `socket` module,
2. create a `socket` object, default is ipv4, tcp.  [see reference](https://docs.python.org/3/library/socket.html#constants)
3. assign an IP address and port number to the socket object using the `bind()` method. `bind()` takes in a tuple of IP address and port number `(ip,port)`. If `ip=''`, then the socket will listen from *any computer* in the network. Take note that port numbers 0 to 1023 should be avoided as they're reserved. In A-Level, you're only required to create a server program that communicates with interacts with a client program on the same computer, so `ip= '127.0.0.1'`, which refers to localhost.
4. The server first creats a queue of n incoming connections by using the `listen(n)`, default is 128.
5. using a while loop, we accept incoming connections to that socket using `accept()` method. `accept()` returns a tuple consisting of :
    - a `connection socket` object
    - a tuple consistent of `(ip_adress, port_number)` of the remote client.
6. with the `connection socket` object from part 5, we can now either:
    - receive message using `recv(n)` method. The parameter n is an integer indicating maximum amount of data to retrieved from the receive buffer. The operation blocks when there are no data in the buffer and returns b'' when the connection is terminated by the sender
    - send message using `send()` method. `send()` takes in `bytes` object as parameters, as such we need to encode the data first using `encode()` method.
7. process the data as needed
8. close the connection by using `close()` method on the connection object.




### Transport Layer: TCP (Tranmission Control Protocol) vs UDP ( User Datagram Protcol)
| Feature                | TCP                                      | UDP                                      |
|------------------------|------------------------------------------|------------------------------------------|
| **Connection**         | Connection-oriented (handshake)          | Connectionless                           |
| **Reliability**        | Guaranteed delivery, ordered             | Best-effort, no guarantees               |
| **Overhead**           | High (ACKs, retransmissions)             | Low                                      |
| **Use Cases**          | Accuracy-critical applications           | Speed-critical applications              |
||Example: Banking APP, Netflix |Example: Voice  |

Choosing between TCP and UDP depends on whether your application prioritizes **reliability** (TCP) or **speed and simplicity** (UDP).

## A level Requires
    - Client Server Architecture
    - Iterative , Single-Threaded Server
    - use TCP as transport

In [ ]:
## Echo Server version 1, just print the message from the Client

from socket import socket

# choose transport layer


#socket(socket.AF_INET, socket.SOCK_STREAM)

# (1) Create a TCP or UDP socket (SOCK_DGRAM for UDP)
#udp_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
listen_socket = socket() ## ipv4, TCP default ## Which Trasport to use TCP or UDP

## (2) bind socket to (IP_Address, Port Number)
listen_socket.bind( ('127.0.0.1', 1234))  # avoid using well-known port number 443, 25

## (3)"Make socket listen for incoming connections
print("Server in listen mode ...")
listen_socket.listen(5) ## create a queue for incoming client connections
## Server processing loop
## while the server is busy running, any incoming client connection will be put in the queue

while True: ## iterative Server
    ## (4) blocks and wait for client connection
    client_socket, remote_address = listen_socket.accept() ## blocking call
    print( f"Accept connection from {remote_address[0]}:{remote_address[1]}")

    # (5) receive from the client buffer or send to client, depending on protocol
    message = client_socket.recv(1024) #?? what shoul be a optimal size
    # (6) Decode and print message
    message = message.decode()
    print(message)
    #(7) Send acknowledgment
    client_socket.sendall(b"OK")
    #(8) Close client connection
    client_socket.close()
    ## How to exit the while loop
    if "bye" in message:
        break

# Terminating the server listenin port
listen_socket.close()
print("Closing listen socket")

##### Echo Server version 2
1.   In version 1 of the Echo server, we are using
        - "OK" to acknowledge receipt of message
        - bye" to terminate server
2.  we do not not when the client message will end or how large the message is, we assume that a single recv(n) will be able to pick up the entire message

Modified with:
1.  using
    -   ACK symbol chr(6) for acknowledgment instead of "OK"
    -   EOT symbol chr(4) for END of transmission instead of "bye"
    -   ETX symbol chr(3) for End of Text
2. handling multiple client sendall(), i.e. message with unknown size


In [ ]:
## Echo Server version 2


from socket import socket
listen_socket = socket()

listen_socket.bind( ('127.0.0.1', 1234))

print("Server in listen mode ...")
listen_socket.listen(5)

while True: ## iterative Server
    client_socket, remote_address = listen_socket.accept() ## blocking call
    print( f"Accept connection from {remote_address[0]}:{remote_address[1]}")
    # how much should I extact from the buffer
    message ="  "
    while True: ## keep extracting from the client buffer until it detected that there is no more data
        chunk = client_socket.recv(10)
        chunk = chunk.decode()
        message += chunk
        if chunk[-1] == chr(3) or chunk[-1] == chr(4):  # chr(3) ETX chr(4) EOT
            break
    print(message)

    #(7) Send acknowledgment
    #client_socket.sendall(b"OK")
    client_socket.sendall(chr(6).encode())

    #(8) Close client connection
    client_socket.close()
    ## Terminate Server
    # if "bye" in message:
    if chr(4) in message:
        break

# Terminating the server listenin port
listen_socket.close()
print("Closing listen socket")

In [ ]:
bin(ord("k"))

In [ ]:
## Modified Server code to handle multiple sends from client

### Client Program
Coding a client program in Python is very similar to making a server program, the difference is that the client program doesn't have to be constantly listening to request. As such, we carry out the following in sequence:
1. import `socket` module,
2. create a `socket` object,

***bind() is optional and is not needed on a client, the socket module will choose the source address and port number for you***

3. connect to the desired IP address and port number through the socket object using the `connect()` method. `connect()` takes in a tuple of IP address and port number `(ip,port)`. For A-Level purposes, we can again set `ip` to be `'127.0.0.1'`
4. send and receive data with `send()` and `receive()` method,
5. close the connection by using `close()` method on the connection object.
6. process the data as needed.


In [ ]:
## You need to run the client code in a seperate notebook
from socket import socket
s = socket()
s.connect(('127.0.0.1', 1234) )
s.sendall("Client2 says Hello ".encode())
ack = s.recv(5)
if ack.decode()== "OK":
    print("message acknowledged")

s.close()


In [ ]:
!python -m idlelib

#### Exercise 1:

Write a Client and Iterative Server "Guess the number game".

Server Code:

1. Server starts and listens for incoming client connection.
2. When the server receives a client connection, it generates a random number from 1 to 100
3. Send a message to the client to guess a number
4. Wait for client to send number
5.      - reply "Correct" and end game if number is correct
        - reply "Low" if number is too small
        - reply "High" if number is too big
6. Repeat 3 till number is guessed correctly and server terminates. Server only needs to serve one client.

Client Code:

1.  Connect to the Server
2.  Wait for the server to start game
3.  Guess the number until the server returns "Correct"

#### Sequence Diagram
![](SequenceDiagram.jpg)

In [ ]:
## Server Code
## Guess Server Code
import socket
import random

listen_socket = socket.socket()
listen_socket.bind(('127.0.0.1', 9999))
listen_socket.listen(1)
print("Guess Server listening ...")
s, addr = listen_socket.accept()
answer = random.randint(1, 100)
print(answer)
guessed = False
s.sendall(b'GUESS') ## initiate conversation
while not guessed :
    #s.send(b'GUESS')
    data = s.recv(1024)
    guess = int(data)
    print("received", guess)
    if guess < answer:
        s.sendall(b'Low')
        #s.send(b'Low')
    elif guess > answer:
        s.sendall(b'High')
        #s.send(b'High')
    else:
        guessed = True
        break
if guessed:
    s.sendall(b'Correct')
    #s.send(b'Correct')
s.close()
print("Server closed")

In [ ]:
### Guess Client Code
## Guess Simplify Client Code (Mr Leong)
import socket, time

s = socket.socket()
s.connect(('127.0.0.1', 9999))

correct = False
while not correct :
    raw = s.recv(1024)
    code = raw.decode()
    print(f"Received {code}")

    if code == 'Low':
        print('Your guess is too low.')
    elif code== 'High':
        print('Your guess is too high.')
    elif code == 'GUESS':
        print("Guess a number between 1 to 100")
    elif code == 'Correct':
        print('You guessed correct!')
        correct = True
        break
    else:
        print(f"{code} is not recognised")
    guess = input('Enter guess (1-100): ')
    s.send(guess.encode())

s.close()


#### Exercise 2:

Write a client and iterative server program for a client to upload a file to the server.

The server will

- listen for incoming client connection
- upon receving a client connection the server will receive the contents of the file and the filename to be saved from the client and then save the file on the server

The client will

- read a file and send its contents and file name to be saved to the server


file_name<1C>xxxxxxxxxxxx<03>

In [ ]:
## Upload Server4
## upload Server
from socket import socket
listen_socket = socket() ## ipv4, TCP
listen_socket.bind(('127.0.0.1', 1234))
listen_socket.listen(5)
print("Server in listen mode...")

while True:
    client_socket, remote_address = listen_socket.accept() ## blocking call
    print( f"Accept connection from {remote_address[0]}:{remote_address[1]}")
    data =""
    while True:
        chunk = client_socket.recv(10).decode()
        data += chunk
        if chunk[-1] == chr(3) or chunk[-1] == chr(4): ## ETX symbol or EOT
            break

    if chunk[-1] == chr(3):
        file_name, content = data.split(chr(0x1c))
        f = open(file_name, "w")
        f.write(content[:-1]) # remove ETX
        f.close()
        client_socket.close()
        print(f"{file_name} uploaded")
    else:
        break

listen_socket.close()
print("Closing listen socket")



Server in listen mode...


In [ ]:
## Upload Client
## Upload Client

from socket import socket
fn = input("Enter file: ")

#f = open("fn")
f = open("HAMLET.txt")
s = socket()
s.connect(('127.0.0.1',1234))

contents = f.read()
f.close()

s.sendall(fn.encode())
s.sendall(chr(0x1c).encode()) # file seperator Delimiter
s.sendall(contents.encode())
s.sendall(chr(3).encode()) # EOF
print("Sent file")
s.close()


---

### Exercise 3 2020/NJC/P2/Q1 H2 Computing
[Link](njc_paper_2020.ipynb)

### Additional Theory and Practical Exercises
[Link](Additional_Exercise.ipynb)